#Importing necessary Libraries and establishing connection requests

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import openpyxl

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests


In [ ]:
HEADERS = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Brave/1.3.361.137'})

In [ ]:
#Sending a request to a URL

URL = "https://www.amazon.in/s?k=support+and+splints&s=review-rank&ds=v1%3A0bI8MAddSL8unb%2FDkFb8nYjiON6GlzLj7%2FXNuQtQH%2Bc&ref=sr_st_review-rank"
webpage = requests.get(URL,headers =HEADERS)

In [ ]:
#Creating a soup of information

soup = BeautifulSoup(webpage.content, "lxml")
soup

#Extracting Titles of the product

In [ ]:
#Discovering the exact tags for Object Extraction
#Extracting titles of the product
title_tag = soup.find_all("span", attrs={"class":'a-size-base-plus a-color-base a-text-normal'})


In [ ]:
# Inner NavigableString Object
#adding a delimiter(seperator)  between every object so that we can split according to it
titles =[]
for x in title_tag :
  titles.append(x.text)
len(titles)

In [ ]:
#converting it to dataframe

df_titles = pd.DataFrame(titles)
df_titles

## extracting total reviews of product



In [ ]:
review_tag = soup.find_all("span", attrs={"class":'a-size-base s-underline-text'})
len(review_tag)

In [ ]:

total_reviews = []
for x in review_tag :
   try:
          total_reviews.append(x.text)

   except AttributeError:
		      total_reviews.append("0")
#another method which we used earlier is-
#rating_value=""
#  for x in rating_tag :
#   rating_value += x.text + " " 
#but with above method which we are using now we dont need to seperate strings by seperators or we dont need to provide delimeters as well like "*" for each string
print(total_reviews)    

In [ ]:
len(total_reviews)

In [ ]:
df_reviews = pd.DataFrame(total_reviews)

In [ ]:
df_product =pd.DataFrame([])
df_product["Title"]=df_titles
df_product['Review_Count'] = df_reviews
df_product

In [ ]:
#Removing ',' and parenthesis from the price
df_product['Review_Count'] = df_product['Review_Count'].str.replace(',', '')
df_product['Review_Count'] = df_product['Review_Count'].str.strip('(' )
df_product['Review_Count'] = df_product['Review_Count'].str.strip(')' )
                     #Removing the empty rows it can also work the same<df[df['B'].str.strip().astype(bool)]>
df_product

In [ ]:
#converting string values of Rating into integer values
df_product["Review_Count"] = df_product["Review_Count"].astype(int)


#Extracting Price of products

In [ ]:
#Extracting Price tag
price_tag = soup.find_all('span' , attrs={"class" :'a-price-whole'})
price_tag

In [ ]:
price_list

In [ ]:
price_list = []
for x in price_tag :
  price_list.append(x.text)
price_list.remove('5,448' )
price_list.remove('4,948')
price_list.remove("3,948")
price_list.remove('3,153')
price_list.remove('2,156.')
price_list.remove('5,573')
price_list.remove('3,192')
price_list.remove('5,448')


len(price_list)

In [ ]:

#Converting this series into dataframe and removing commas and fullstops
df_prices = pd.DataFrame(price_list)
df_prices[0] = df_prices[0].str.replace(",","")      
df_prices[0] = df_prices[0].str.replace(".","") 
df_prices

In [ ]:
#Converting strings into integers
df_prices[0] = df_prices[0].astype(int)

In [ ]:
#Adding prices into our final product dataframe
df_product['Price'] = df_prices[0]

In [ ]:
df_product

#Extracting Ratings of products

In [ ]:
rating_tag= soup.find_all('span', attrs={'class':"a-icon-alt"})
len(rating_tag)
rating_tag

In [ ]:
rating_list= [ ]
for x in rating_tag:
   rating_list.append(x.text)
len(rating_list)

In [ ]:
rating_list

In [ ]:
rating_list = rating_list[2: -4]

df_list = pd.DataFrame(rating_list)
df_list1 = df_list[0].str.split(' ', expand=True)

In [ ]:
df_list1=df_list1.drop( [1,2,3,4] , axis = 1)         #dropping unnecessary columns

In [ ]:
df_list1[0].astype(float)
df_product['Rating'] = df_list1[0]

In [ ]:
df_product

#Extracting Brand Name

In [ ]:
#Function to extract brand tags
def brand(soup) :
   Brand_tag= soup.find('a', attrs={'id':"bylineInfo"})
   return Brand_tag



In [ ]:
#fetching tags of all the product pages
link_tag= soup.find_all('a', attrs={'class':"a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"})


In [ ]:
#extracting the href element from tags
links_list= [ ]
for link in link_tag:
	links_list.append(link.get('href'))
 
links_list

In [ ]:
brand_list=[]
for link in links_list:
  new_webpage = requests.get("https://www.amazon.in" + link, headers=HEADERS)
  new_soup = BeautifulSoup(new_webpage.content, "lxml")
  brand_list.append(brand(new_soup))


In [ ]:
#Extracting text element, that is brand name from the brand tags
all_brand=[]
for x in brand_list:
  try:
     all_brand.append(x.text)
  except AttributeError:
      all_brand.append("")                      #Replacing NONE values(if any) with ""

In [ ]:
#Removing "" (empty elements) from list
bran=[]
for x in all_brand :
  if x!= '':
    bran.append(x)

print(bran)                      #Final List of all the brands of the products

#Saving Data Frame in Excel Format

In [ ]:
writer = pd.ExcelWriter('support_and_splints2.xlsx')                #need openpyxl library for this
df_product.to_excel(writer)
writer.save()

